### Packages loading

In [1]:
import numpy as np
import random
import qiskit
import qtm.evolution
import qtm.state
import qtm.qcompilation
import qtm.ansatz
import qtm.constant
from qtm.evolution import environment, mutate, selection, crossover
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2


## 1. Quantum architecture search for quantum state preparation problem

Now, we defind the problem which is need to solve

In [3]:

def VQE_fitness(qc: qiskit.QuantumCircuit):
    # Code here
    return 


Main algorithm, includes: init population, selection, crossover, mutate.

In [21]:
params = {'depth': 10,
          'num_individual': 8,  # Must mod 8 = 0
          'num_generation': 30,
          'num_qubits': 3,
          'threshold': 0.2,
          'prob_mutate': 0.01}

env = environment.EEnvironment(
    params,
    fitness_func = VQE_fitness,
    selection_func = selection.elitist_selection,
    crossover_func= crossover.onepoint_crossover,
    mutate_func=mutate.bitflip_mutate,
    pool = qtm.constant.operations
)

env.initialize_population()
env.evol() 

Step: 100%|██████████| 30/30 [22:45<00:00, 45.52s/it]

End best score, end evol progress, percent target: 0.6
